In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
from shapely.geometry import Point

from mpl_toolkits.axes_grid1 import make_axes_locatable
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from sklearn.neighbors import DistanceMetric
from math import radians

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


from sklearn import svm
from sklearn import metrics


from sklearn.datasets import make_moons, make_blobs
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, classification_report, make_scorer, recall_score, f1_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier

In [2]:
# Utilisation du DF data_features qui contient toute les données nettoyées et le climat koppen intégré

df = pd.read_csv('data_features.csv', sep=',', index_col=0) 
df.head()

,sunshine,windgustspeed,humidity3pm,pressure3pm,year,windspeed3pm,temp_amplitude,cloud3pm,rainfall,climat,...,cloud9am,winddir9am_sin,latitude,month,geography,raintoday,evaporation,windspeed9am,windgustdir_cos,raintomorrow
0,10.9,44.0,22.0,1007.1,2008,24.0,9.5,7.0,0.6,0,...,8.0,1.224647e-16,-36.065766,12,0,0,7.6,20.0,-1.000000,0
1,12.9,44.0,25.0,1007.8,2008,22.0,17.7,1.0,0.0,0,...,1.0,9.238795e-01,-36.065766,12,0,0,12.0,4.0,-0.923880,0
2,8.0,46.0,30.0,1008.7,2008,26.0,12.8,2.0,0.0,0,...,7.0,1.224647e-16,-36.065766,12,0,0,8.0,19.0,-0.923880,0
3,2.2,24.0,16.0,1012.8,2008,9.0,18.8,6.0,0.0,0,...,8.0,-7.071068e-01,-36.065766,12,0,0,10.0,11.0,0.707107,0
4,6.7,41.0,33.0,1006.0,2008,20.0,14.8,8.0,1.0,0,...,7.0,3.826834e-01,-36.065766,12,0,0,4.8,7.0,-1.000000,0


In [3]:
features = ["sunshine","humidity3pm","pressure3pm","windspeed3pm","temp_amplitude",
            "cloud3pm","rainfall","climat","winddir3pm_sin","cloud9am", "latitude", 
            "geography", "raintoday", "evaporation", "windgustdir_cos", "windgustspeed", "humidity9am", "winddir3pm_sin",
           "longitude", "winddir9am_cos", "winddir9am_sin", "windspeed9am"]

In [4]:
#On supprime toutes les observations avec des NA
df1 = df.dropna()

target = df1["raintomorrow"]
data = df1[features]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=123)

In [6]:

from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

# Définition de l'espace de recherche pour l'optimisation bayésienne
space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 15),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Real(0.1, 1.0, prior='uniform')
}


In [7]:
# Définition du modèle à optimiser (Random Forest)
model = RandomForestClassifier()

In [8]:
# Définition de l'optimiseur bayésien avec 20 itérations
opt = BayesSearchCV(
    model,
    space,
    n_iter=20,
    scoring='accuracy',
    n_jobs=-1,
    cv=5
)

# Lancement de l'optimisation
opt.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
              scoring='accuracy',
              search_spaces={'max_depth': Integer(low=3, high=15, prior='uniform', transform='normalize'),
                             'max_features': Real(low=0.1, high=1.0, prior='uniform', transform='normalize'),
                             'min_samples_leaf': Integer(low=1, high=5, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=10, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=1000, prior='uniform', transform='normalize')})

In [11]:
# Affichage des meilleurs hyperparamètres trouvés
print('Meilleurs hyperparamètres :', opt.best_params_)

# Évaluation du modèle avec les hyperparamètres trouvés
score_1 = opt.score(X_test, y_test)
print('Score de l\'optimisation bayésienne : {:.2f}'.format(score_1))

Meilleurs hyperparamètres : OrderedDict([('max_depth', 15), ('max_features', 0.4281218178874725), ('min_samples_leaf', 1), ('min_samples_split', 9), ('n_estimators', 475)])
Score de l'optimisation bayésienne : 0.88


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier


# Définition de l'espace de recherche pour l'optimisation bayésienne
space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 15),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Real(0.1, 1.0, prior='uniform')
}

# Définition du modèle à optimiser (ExtraTreesClassifier)
model = ExtraTreesClassifier()

# Définition de l'optimiseur bayésien avec 20 itérations
opt = BayesSearchCV(
    model,
    space,
    n_iter=20,
    scoring='accuracy',
    n_jobs=-1,
    cv=5
)

# Lancement de l'optimisation
opt.fit(X_train, y_train)


In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

# Définition de l'espace de recherche pour l'optimisation bayésienne
space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 15),
    'learning_rate': Real(0.01, 1.0, prior='log-uniform'),
    'num_leaves': Integer(20, 200),
    'min_child_samples': Integer(5, 50),
    'subsample': Real(0.1, 1.0, prior='uniform'),
    'colsample_bytree': Real(0.1, 1.0, prior='uniform')
}

# Définition du modèle à optimiser (LGBMClassifier)
model = LGBMClassifier()

# Définition de l'optimiseur bayésien avec 20 itérations
opt = BayesSearchCV(
    model,
    space,
    n_iter=20,
    scoring='accuracy',
    n_jobs=-1,
    cv=5
)

# Lancement de l'optimisation
opt.fit(X_train, y_train)
